# Import required modules

In [1]:
import os 
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages')
print(sys.path)

['/Users/seojiwoo/Desktop/URECA_May', '/Users/seojiwoo/.vscode/extensions/ms-toolsai.jupyter-2022.6.1101950301/pythonFiles', '/Users/seojiwoo/.vscode/extensions/ms-toolsai.jupyter-2022.6.1101950301/pythonFiles/lib/python', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/seojiwoo/Desktop/URECA_May/SISSO/lib/python3.10/site-packages', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages']


In [2]:
import os
import pandas as pd
import numpy as np
from itertools import combinations, product
from time import time
import matplotlib.pyplot as plt 
from sklearn.linear_model import Lasso
import scipy.stats as ss
import warnings
from collections import Counter
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from IPython.display import HTML
from jupyter_jsmol import JsmolView
import pathlib
import plotly #by adding the path in sys.path 


from compressed_sensing.units import J 

import nglview

from compressed_sensing.sisso import SissoRegressor
from compressed_sensing.Alter_combine_features import combine_features
from compressed_sensing.visualizer import Visualizer
from compressed_sensing.scatter_plot import  show_scatter_plot
# from sissopp import Inputs, FeatureSpace, SISSORegressor, FeatureNode, Unit
# from sissopp.py_interface import read_csv
# from sissopp.py_interface.import_dataframe import get_unit


# set display options for the notebook 
# %matplotlib inline
warnings.filterwarnings('ignore')

#bokeh works perfectly fine with jupyter notebook, not VS code. why? 

Loading BokehJS ...

### SISSO with E-hull (new + original operator, LOOCV & K-Fold)

In [3]:
#import Data
# selected_feature_list = ['r_s', 'r_p', 'r_d', 'EA', 'IP']
# selected_feature_list = ordered_selected_features[:7] 


df_2 = pd.read_csv("275_with_EHull.csv") #need to check its location later)
df_2.dropna(inplace=True)
df_2.columns

ordered_selected_features = ['HoF','Length(x)', 'E_Hull','r-(M)', 'r-(X)', 'r-(T)', 'IP(M)', 'IP(X)', 'IP(T)', 'EN(M)', 'EN(X)','EN(T)', 'EA(M)', 'EA(X)', 'EA(T)']
df_2[ordered_selected_features]

def get_data(selected_feature_list, allowed_operations):
    
    # extract work function and selected features from df 
    P = df_2['Workfunction'].values
    df_features = df_2.iloc[:, 5:][selected_feature_list] #select 5th column onwards 
    print("df_features.columns", df_features.columns)
    
    # derive new features using allowed_operations
    df_combined = combine_features(df=df_features, allowed_operations=allowed_operations) 
    return P, df_combined


selected_feature_list = df_2.columns[5:]
allowed_operations = ['+','-','|-|','^3','^2','sin','cos','tan'] #exp,division seems to be the problem. because zero value cannot. 
P, df_D = get_data(selected_feature_list, allowed_operations)
features_list = df_D.columns.tolist()
chemical_formulas = df_D.index.tolist()
D = df_D.values
# D=a[a != 0] #would this disrupt the result actually? 

features_list = df_D.columns.tolist()

#look at df_D for the created dataset

df_features.columns Index(['HoF', 'E_Hull', 'Length(x)', 'r-(M)', 'r-(X)', 'r-(T)', 'IP(M)',
       'IP(X)', 'IP(T)', 'EN(M)', 'EN(X)', 'EN(T)', 'EA(M)', 'EA(X)', 'EA(T)'],
      dtype='object')
Selected operations:
 ['+', '-', '|-|', '^3', '^2', 'sin', 'cos', 'tan']
Number of total features generated: 121


In [4]:
sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)

sisso.fit(D, P) #error 
sisso.print_models(features_list)


          RMSE            Model
1D:	1.387989	+ 1.321 |EN(X)-EN(T)| + 3.179
2D:	1.281586	+ 1.534 |EN(X)-EN(T)| + 0.550 (EN(M)+EN(T)) + 0.502
3D:	1.061405	+ 1.890 |EN(X)-EN(T)| + 1.174 (EN(M)+EN(T)) + 0.775 |IP(M)-EA(T)| - 6.683
4D:	0.865174	+ 2.143 |EN(X)-EN(T)| + 2.096 (EN(M)+EN(T)) + 1.607 |IP(M)-EA(T)| - 1.973 IP(M) - 1.665
5D:	0.808244	+ 2.245 |EN(X)-EN(T)| + 2.390 (EN(M)+EN(T)) + 1.806 |IP(M)-EA(T)| - 1.922 IP(M) + 0.953 |EN(M)-EN(X)| - 5.613
6D:	0.757767	+ 2.118 |EN(X)-EN(T)| + 2.514 (EN(M)+EN(T)) + 1.718 |IP(M)-EA(T)| - 1.831 IP(M) + 1.121 |EN(M)-EN(X)| - 0.020 |r-(M)-r-(T)| - 6.063
7D:	0.727480	+ 2.095 |EN(X)-EN(T)| + 2.618 (EN(M)+EN(T)) + 1.660 |IP(M)-EA(T)| - 1.916 IP(M) + 1.290 |EN(M)-EN(X)| - 0.020 |r-(M)-r-(T)| + 1.362 sin(E_Hull) - 6.222
8D:	0.694519	+ 2.032 |EN(X)-EN(T)| + 2.874 (EN(M)+EN(T)) + 1.760 |IP(M)-EA(T)| - 1.848 IP(M) + 1.453 |EN(M)-EN(X)| - 0.024 |r-(M)-r-(T)| + 1.812 sin(E_Hull) - 0.715 (EN(M)+EN(X)) - 5.442
9D:	0.675483	+ 2.037 |EN(X)-EN(T)| + 2.914 (EN(M)+EN

In [5]:
# Leave-one-out cross-validation
n_compounds = len(P)
dimensions = range(1, 11) #(1,2,3,4)
features_count = [[] for i in range(10)] #what exactly does this do? 
P_predict = np.empty([len(dimensions), n_compounds]) #create a empty array with no input. (row*column)
list_1D = [] 
list_2D = [] 
list_3D = [] 
list_4D = [] 
list_5D = []
list_6D = []
list_7D = []
list_8D = []
list_9D = []
list_10D = []

sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)
loo = LeaveOneOut() #cross validation method 
#examples of loo. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html 


for indices_train, index_test in loo.split(P):
    i_cv = index_test[0] 
    print('%2s) Leave out %s: Ediff_ref = %.2f eV' 
          % (index_test[0]+1, chemical_formulas[i_cv], P[i_cv])) #this is called the printf format string. https://en.wikipedia.org/wiki/Printf_format_string
        
    sisso.fit(D[indices_train], P[indices_train])
    sisso.print_models(features_list)    
    
    for dim in dimensions:      
        features = [features_list[i] for i in sisso.l0_selected_indices[dim - 1]]
        predicted_value = sisso.predict(D[index_test], dim=dim)[0]
        
        features_count[dim-1].append(tuple(features))        
        P_predict[dim-1, i_cv] = predicted_value

        if dim == 1: 
            list_1D.append(P[index_test]-predicted_value)

        if dim == 2: 
            list_2D.append(P[index_test]-predicted_value)

        if dim == 3: 
            list_3D.append(P[index_test]-predicted_value)

        if dim == 4: 
            list_4D.append(P[index_test]- predicted_value)

        if dim == 5: 
            list_5D.append(P[index_test]- predicted_value)

        if dim == 6: 
            list_6D.append(P[index_test]-predicted_value)

        if dim == 7: 
            list_7D.append(P[index_test]-predicted_value)

        if dim == 8: 
            list_8D.append(P[index_test]-predicted_value)

        if dim == 9: 
            list_9D.append(P[index_test]- predicted_value)

        if dim == 10: 
            list_10D.append(P[index_test]- predicted_value)

        print('Ediff_predicted(%sD) = %.3f eV' %(dim, predicted_value))
    print('-----') #unit for workfunction is eV. change that later. 

for dimensions in dim: 
    sum()

 1) Leave out 0: Ediff_ref = 2.27 eV
          RMSE            Model
1D:	1.388402	+ 1.310 |EN(X)-EN(T)| + 3.193
2D:	1.282737	+ 1.525 |EN(X)-EN(T)| + 0.547 (EN(M)+EN(T)) + 0.523
3D:	1.063218	+ 1.887 |EN(X)-EN(T)| + 1.172 (EN(M)+EN(T)) + 0.774 |IP(M)-EA(T)| - 6.668
4D:	0.866701	+ 2.145 |EN(X)-EN(T)| + 2.098 (EN(M)+EN(T)) + 1.608 |IP(M)-EA(T)| - 1.974 IP(M) - 1.670
5D:	0.809459	+ 2.249 |EN(X)-EN(T)| + 2.393 (EN(M)+EN(T)) + 1.809 |IP(M)-EA(T)| - 1.925 IP(M) + 0.955 |EN(M)-EN(X)| - 5.635
6D:	0.757820	+ 2.126 |EN(X)-EN(T)| + 2.524 (EN(M)+EN(T)) + 1.723 |IP(M)-EA(T)| - 1.835 IP(M) + 1.128 |EN(M)-EN(X)| - 0.020 |r-(M)-r-(T)| - 6.116
7D:	0.727069	+ 2.103 |EN(X)-EN(T)| + 2.630 (EN(M)+EN(T)) + 1.665 |IP(M)-EA(T)| - 1.922 IP(M) + 1.299 |EN(M)-EN(X)| - 0.021 |r-(M)-r-(T)| + 1.370 sin(E_Hull) - 6.282
8D:	0.694657	+ 2.039 |EN(X)-EN(T)| + 2.881 (EN(M)+EN(T)) + 1.763 |IP(M)-EA(T)| - 1.853 IP(M) + 1.458 |EN(M)-EN(X)| - 0.024 |r-(M)-r-(T)| + 1.814 sin(E_Hull) - 0.709 (EN(M)+EN(X)) - 5.497
9D:	0.675630	+ 

TypeError: 'int' object is not iterable

In [6]:
list_1D = [((element)**2) for element in list_1D] #this is the equation for RMSE 
list_2D = [((element)**2) for element in list_2D] 
list_3D = [((element)**2) for element in list_3D] 
list_4D = [((element)**2) for element in list_4D] 
list_5D = [((element)**2) for element in list_5D] 
list_6D = [((element)**2) for element in list_6D] 
list_7D = [((element)**2) for element in list_7D] 
list_8D = [((element)**2) for element in list_8D] 
list_9D = [((element)**2) for element in list_9D] 
list_10D = [((element)**2) for element in list_10D] 

def Average(lst):
    return sum(lst) / len(lst)


print(Average(list_1D),Average(list_2D), Average(list_3D), Average(list_4D),Average(list_5D),Average(list_6D),Average(list_7D), Average(list_8D), Average(list_9D),Average(list_10D)) 
#rmse value for 1D-4D. 

[1.95411303] [1.67131877] [1.15281979] [0.77552334] [0.68398583] [0.62861485] [0.65900736] [0.59150163] [0.59101906] [0.51433105]


In [7]:
##Plot Prediction errors for loocv 
prediction_errors = np.linalg.norm(P-P_predict, axis=1)/np.sqrt(n_compounds)
xs = [P] *4
ys = P_predict
legend = ['%sD, RMSE = %.4f eV' %(dim, prediction_errors[dim-1]) for dim in dimensions]
data_point_labels = [df_D.index.tolist()]*4

show_scatter_plot(xs, ys, data_point_labels=data_point_labels, 
                  x_label='WF_DFT', y_label='WF_predicted', legend=legend, unit='eV')

In [8]:
# Print descriptor selection frequency for loocv 
print("Descriptor selection frequency")
for dim in dimensions:    
    df_frequency = pd.DataFrame( Counter(features_count[dim-1]).most_common(10), columns=['Features', 'Frequency'] )
    print('-----------------\n%sD:\n%s' % (dim, df_frequency))

# create table to display errors and models
feat = np.array(features_count).flatten('F')
Pred = np.array(P_predict).flatten('F')
Pred_errors = np.abs(P-P_predict).flatten('F')
Ref_values = [r for p in P for r in [p,p,p] ]
Mats = [m for mat in chemical_formulas for m in [mat, mat, mat] ]
Dims = ['1D','2D','3D'] * n_compounds

df_loo = pd.DataFrame(list(zip(Ref_values,Pred,Pred_errors,feat)), index = [Mats,Dims],
                  columns=['P_ref[eV]', 'P_pred[eV]', 'abs. error [eV]', 'Selected features'])

#  if you do not want to sort the data frame by the prediction error comment out the next line 
df_loo = df_loo.sort_values('abs. error [eV]', ascending=False)
pd.set_option('display.expand_frame_repr', False)

display(df_loo)

#but it's not that accurate of a prediction model... can be very wonky.

Descriptor selection frequency
-----------------
1D:
           Features  Frequency
0  (|EN(X)-EN(T)|,)        275
-----------------
2D:
                         Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)))        275
-----------------
3D:
                                        Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|)        248
1  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)))         27
-----------------
4D:
                                            Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...        248
1  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...         27
-----------------
5D:
                                            Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...        128
1  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...        120
2  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...         14
3  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...         13
---------------

P_ref[eV]  P_pred[eV]  abs. error [eV]                                  Selected features
270 1D   1.347678    3.687384         3.129689                                   (|EN(X)-EN(T)|,)
    2D   1.347678    3.897748         2.919325                     (|EN(X)-EN(T)|, (EN(M)+EN(T)))
39  3D   5.664074    5.640648         2.915745  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
    2D   5.664074    5.634446         2.909543  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
    1D   5.664074    5.587632         2.862728  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
...           ...         ...              ...                                                ...
72  3D   2.324315    2.335403         0.004613  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
205 1D   1.561346    4.727063         0.003466  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...
116 1D   5.004303    3.579491         0.003029  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
125 1D   3.314447    5.215839         0.002146  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
215 1D   3.947450    1.423338         0.000330  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...

[825 rows x 4 columns]

In [9]:
# K-fold cross-validation (this is messy. work in progress)  
n_compounds = len(P)
dimensions = range(1, 11)
features_count = [[] for i in range(10)]
P_predict = np.empty([len(dimensions), n_compounds])
list_K_1D = [] 
list_K_2D = [] 
list_K_3D = [] 
list_K_4D = [] 
list_K_5D = []
list_K_6D = []
list_K_7D = []
list_K_8D = []
list_K_9D = []
list_K_10D = []

#insert empty list here 

sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)
kfold = KFold() #i think the default setting is: kfold = KFold(n_splits=5, shuffle=False, random_state=None)

for indices_train, index_test in kfold.split(P):
    i_cv = index_test[0]
    print('%2s) Leave out %s: Ediff_ref = %.2f eV' 
          % (index_test[0]+1, chemical_formulas[i_cv], P[i_cv])) #chemical_formulas = df_D.index.tolist() (list of values ranging from 0 to 274)
        
    sisso.fit(D[indices_train], P[indices_train])
    sisso.print_models(features_list)    
    
    for dim in dimensions:      
        features = [features_list[i] for i in sisso.l0_selected_indices[dim - 1]] #list index out of range 
        predicted_value = sisso.predict(D[index_test], dim=dim)[0]
        
        features_count[dim-1].append( tuple(features) )        
        P_predict[dim-1, i_cv] = predicted_value

        if dim == 1: 
            list_K_1D.append(P[index_test]-predicted_value)

        if dim == 2: 
            list_K_2D.append(P[index_test]-predicted_value)

        if dim == 3: 
            list_K_3D.append(P[index_test]-predicted_value)

        if dim == 4: 
            list_K_4D.append(P[index_test]- predicted_value)

        if dim == 5: 
            list_K_5D.append(P[index_test]- predicted_value)

        if dim == 6: 
            list_K_6D.append(P[index_test]-predicted_value)

        if dim == 7: 
            list_K_7D.append(P[index_test]-predicted_value)

        if dim == 8: 
            list_K_8D.append(P[index_test]-predicted_value)

        if dim == 9: 
            list_K_9D.append(P[index_test]- predicted_value)

        if dim == 10: 
            list_K_10D.append(P[index_test]- predicted_value)
        
        print('Ediff_predicted(%sD) = %.3f eV' %(dim, predicted_value))
    print('-----') #unit for workfunction is eV. change that later. 

for dimensions in dim: 
    sum()


 1) Leave out 0: Ediff_ref = 2.27 eV
          RMSE            Model
1D:	1.410455	+ 1.342 |EN(X)-EN(T)| + 3.167
2D:	1.299196	+ 1.563 |EN(X)-EN(T)| + 0.562 (EN(M)+EN(T)) + 0.433
3D:	1.023251	+ 1.867 |EN(X)-EN(T)| + 1.425 (EN(M)+EN(T)) + 1.065 (EA(M)-EA(T)) - 2.295
4D:	0.900414	+ 2.098 |EN(X)-EN(T)| + 1.975 (EN(M)+EN(T)) + 1.533 (EA(M)-EA(T)) - 1.722 EA(M) - 3.660
5D:	0.805303	+ 2.243 |EN(X)-EN(T)| + 2.406 (EN(M)+EN(T)) + 1.842 (EA(M)-EA(T)) - 1.566 EA(M) - 1.237 (EN(M)-EN(X)) - 6.870
6D:	0.750189	+ 2.072 |EN(X)-EN(T)| + 2.542 (EN(M)+EN(T)) + 1.762 (EA(M)-EA(T)) - 1.258 EA(M) - 1.482 (EN(M)-EN(X)) - 0.021 |r-(M)-r-(T)| - 7.443
7D:	0.714959	+ 2.010 |EN(X)-EN(T)| + 2.802 (EN(M)+EN(T)) + 1.904 (EA(M)-EA(T)) - 1.079 EA(M) - 1.609 (EN(M)-EN(X)) - 0.027 |r-(M)-r-(T)| - 0.726 (EN(M)+EN(X)) - 5.311
8D:	0.663403	+ 1.961 |EN(X)-EN(T)| + 3.015 (EN(M)+EN(T)) + 1.854 (EA(M)-EA(T)) - 1.125 EA(M) - 1.848 (EN(M)-EN(X)) - 0.029 |r-(M)-r-(T)| - 1.039 (EN(M)+EN(X)) + 1.803 sin(E_Hull) - 5.714
9D:	0.618545	

TypeError: 'int' object is not iterable

### SISSO with E-hull (original operator, LOOCV & K-Fold)

In [10]:
#import Data
# selected_feature_list = ['r_s', 'r_p', 'r_d', 'EA', 'IP']
# selected_feature_list = ordered_selected_features[:7] 


df_2 = pd.read_csv("275_with_EHull.csv") #need to check its location later)
df_2.dropna(inplace=True)
df_2.columns

ordered_selected_features = ['HoF','Length(x)', 'E_Hull','r-(M)', 'r-(X)', 'r-(T)', 'IP(M)', 'IP(X)', 'IP(T)', 'EN(M)', 'EN(X)','EN(T)', 'EA(M)', 'EA(X)', 'EA(T)']
df_2[ordered_selected_features]

def get_data(selected_feature_list, allowed_operations):
    
    # extract work function and selected features from df 
    P = df_2['Workfunction'].values
    df_features = df_2.iloc[:, 5:][selected_feature_list] #select 5th column onwards 
    print("df_features.columns", df_features.columns)
    
    # derive new features using allowed_operations
    df_combined = combine_features(df=df_features, allowed_operations=allowed_operations) 
    return P, df_combined


selected_feature_list = df_2.columns[5:]
allowed_operations = ['+','-','|-|','^3','^2'] #exp,division seems to be the problem. because zero value cannot. 
P, df_D = get_data(selected_feature_list, allowed_operations)
features_list = df_D.columns.tolist()
chemical_formulas = df_D.index.tolist()
D = df_D.values
# D=a[a != 0] #would this disrupt the result actually? 

features_list = df_D.columns.tolist()

#look at df_D for the created dataset

df_features.columns Index(['HoF', 'E_Hull', 'Length(x)', 'r-(M)', 'r-(X)', 'r-(T)', 'IP(M)',
       'IP(X)', 'IP(T)', 'EN(M)', 'EN(X)', 'EN(T)', 'EA(M)', 'EA(X)', 'EA(T)'],
      dtype='object')
Selected operations:
 ['+', '-', '|-|', '^3', '^2']
Number of total features generated: 115


In [11]:
sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)

sisso.fit(D, P) #error 
sisso.print_models(features_list)


          RMSE            Model
1D:	1.387989	+ 1.321 |EN(X)-EN(T)| + 3.179
2D:	1.281586	+ 1.534 |EN(X)-EN(T)| + 0.550 (EN(M)+EN(T)) + 0.502
3D:	1.061405	+ 1.890 |EN(X)-EN(T)| + 1.174 (EN(M)+EN(T)) + 0.775 (IP(M)-EA(T)) - 6.683
4D:	0.865174	+ 2.143 |EN(X)-EN(T)| + 2.096 (EN(M)+EN(T)) + 1.607 (IP(M)-EA(T)) - 1.973 IP(M) - 1.665
5D:	0.808244	+ 2.245 |EN(X)-EN(T)| + 2.390 (EN(M)+EN(T)) + 1.806 (IP(M)-EA(T)) - 1.922 IP(M) + 0.953 |EN(M)-EN(X)| - 5.613
6D:	0.757767	+ 2.118 |EN(X)-EN(T)| + 2.514 (EN(M)+EN(T)) + 1.718 (IP(M)-EA(T)) - 1.831 IP(M) + 1.121 |EN(M)-EN(X)| - 0.020 |r-(M)-r-(T)| - 6.063
7D:	0.729321	+ 2.091 |EN(X)-EN(T)| + 2.610 (EN(M)+EN(T)) + 1.664 (IP(M)-EA(T)) - 1.914 IP(M) + 1.276 |EN(M)-EN(X)| - 0.020 |r-(M)-r-(T)| + 1.152 E_Hull - 6.158
8D:	0.697848	+ 2.029 |EN(X)-EN(T)| + 2.858 (EN(M)+EN(T)) + 1.763 (IP(M)-EA(T)) - 1.847 IP(M) + 1.429 |EN(M)-EN(X)| - 0.024 |r-(M)-r-(T)| + 1.524 E_Hull - 0.698 (EN(M)+EN(X)) - 5.377
9D:	0.672132	+ 1.918 |EN(X)-EN(T)| + 3.063 (EN(M)+EN(T)) + 1.7

In [12]:
# Leave-one-out cross-validation
n_compounds = len(P)
dimensions = range(1, 11) #(1,2,3,4)
features_count = [[] for i in range(10)] #what exactly does this do? 
P_predict = np.empty([len(dimensions), n_compounds]) #create a empty array with no input. (row*column)
list_1D = [] 
list_2D = [] 
list_3D = [] 
list_4D = [] 
list_5D = []
list_6D = []
list_7D = []
list_8D = []
list_9D = []
list_10D = []

sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)
loo = LeaveOneOut() #cross validation method 
#examples of loo. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html 


for indices_train, index_test in loo.split(P):
    i_cv = index_test[0] 
    print('%2s) Leave out %s: Ediff_ref = %.2f eV' 
          % (index_test[0]+1, chemical_formulas[i_cv], P[i_cv])) #this is called the printf format string. https://en.wikipedia.org/wiki/Printf_format_string
        
    sisso.fit(D[indices_train], P[indices_train])
    sisso.print_models(features_list)    
    
    for dim in dimensions:      
        features = [features_list[i] for i in sisso.l0_selected_indices[dim - 1]]
        predicted_value = sisso.predict(D[index_test], dim=dim)[0]
        
        features_count[dim-1].append(tuple(features))        
        P_predict[dim-1, i_cv] = predicted_value

        if dim == 1: 
            list_1D.append(P[index_test]-predicted_value)

        if dim == 2: 
            list_2D.append(P[index_test]-predicted_value)

        if dim == 3: 
            list_3D.append(P[index_test]-predicted_value)

        if dim == 4: 
            list_4D.append(P[index_test]- predicted_value)

        if dim == 5: 
            list_5D.append(P[index_test]- predicted_value)

        if dim == 6: 
            list_6D.append(P[index_test]-predicted_value)

        if dim == 7: 
            list_7D.append(P[index_test]-predicted_value)

        if dim == 8: 
            list_8D.append(P[index_test]-predicted_value)

        if dim == 9: 
            list_9D.append(P[index_test]- predicted_value)

        if dim == 10: 
            list_10D.append(P[index_test]- predicted_value)

        print('Ediff_predicted(%sD) = %.3f eV' %(dim, predicted_value))
    print('-----') #unit for workfunction is eV. change that later. 

for dimensions in dim: 
    sum()

 1) Leave out 0: Ediff_ref = 2.27 eV
          RMSE            Model
1D:	1.388402	+ 1.310 |EN(X)-EN(T)| + 3.193
2D:	1.282737	+ 1.525 |EN(X)-EN(T)| + 0.547 (EN(M)+EN(T)) + 0.523
3D:	1.063218	+ 1.887 |EN(X)-EN(T)| + 1.172 (EN(M)+EN(T)) + 0.774 (IP(M)-EA(T)) - 6.668
4D:	0.866701	+ 2.145 |EN(X)-EN(T)| + 2.098 (EN(M)+EN(T)) + 1.608 (IP(M)-EA(T)) - 1.974 IP(M) - 1.670
5D:	0.809459	+ 2.249 |EN(X)-EN(T)| + 2.393 (EN(M)+EN(T)) + 1.809 (IP(M)-EA(T)) - 1.925 IP(M) + 0.955 |EN(M)-EN(X)| - 5.635
6D:	0.757820	+ 2.126 |EN(X)-EN(T)| + 2.524 (EN(M)+EN(T)) + 1.723 (IP(M)-EA(T)) - 1.835 IP(M) + 1.128 |EN(M)-EN(X)| - 0.020 |r-(M)-r-(T)| - 6.116
7D:	0.728948	+ 2.100 |EN(X)-EN(T)| + 2.622 (EN(M)+EN(T)) + 1.669 (IP(M)-EA(T)) - 1.920 IP(M) + 1.285 |EN(M)-EN(X)| - 0.020 |r-(M)-r-(T)| + 1.158 E_Hull - 6.218
8D:	0.698014	+ 2.036 |EN(X)-EN(T)| + 2.864 (EN(M)+EN(T)) + 1.766 (IP(M)-EA(T)) - 1.852 IP(M) + 1.435 |EN(M)-EN(X)| - 0.024 |r-(M)-r-(T)| + 1.526 E_Hull - 0.692 (EN(M)+EN(X)) - 5.431
9D:	0.672144	+ 1.926 |EN(

TypeError: 'int' object is not iterable

In [13]:
list_1D = [((element)**2) for element in list_1D] #this is the equation for RMSE 
list_2D = [((element)**2) for element in list_2D] 
list_3D = [((element)**2) for element in list_3D] 
list_4D = [((element)**2) for element in list_4D] 
list_5D = [((element)**2) for element in list_5D] 
list_6D = [((element)**2) for element in list_6D] 
list_7D = [((element)**2) for element in list_7D] 
list_8D = [((element)**2) for element in list_8D] 
list_9D = [((element)**2) for element in list_9D] 
list_10D = [((element)**2) for element in list_10D] 

def Average(lst):
    return sum(lst) / len(lst)


print(Average(list_1D),Average(list_2D), Average(list_3D), Average(list_4D),Average(list_5D),Average(list_6D),Average(list_7D), Average(list_8D), Average(list_9D),Average(list_10D)) 
#rmse value for 1D-10D

[1.95411303] [1.67131877] [1.15281979] [0.77552334] [0.68398583] [0.62861485] [0.71135596] [0.59286824] [0.52911935] [0.47012233]


In [14]:
#Plot Prediction errors
prediction_errors = np.linalg.norm(P-P_predict, axis=1)/np.sqrt(n_compounds)
xs = [P] *4
ys = P_predict
legend = ['%sD, RMSE = %.4f eV' %(dim, prediction_errors[dim-1]) for dim in dimensions]
data_point_labels = [df_D.index.tolist()]*4

show_scatter_plot(xs, ys, data_point_labels=data_point_labels, 
                  x_label='WF_DFT', y_label='WF_predicted', legend=legend, unit='eV')

In [15]:
# Print descriptor selection frequency
print("Descriptor selection frequency")
for dim in dimensions:    
    df_frequency = pd.DataFrame( Counter(features_count[dim-1]).most_common(10), columns=['Features', 'Frequency'] )
    print('-----------------\n%sD:\n%s' % (dim, df_frequency))

# create table to display errors and models
feat = np.array(features_count).flatten('F')
Pred = np.array(P_predict).flatten('F')
Pred_errors = np.abs(P-P_predict).flatten('F')
Ref_values = [r for p in P for r in [p,p,p] ]
Mats = [m for mat in chemical_formulas for m in [mat, mat, mat] ]
Dims = ['1D','2D','3D'] * n_compounds

df_loo = pd.DataFrame(list(zip(Ref_values,Pred,Pred_errors,feat)), index = [Mats,Dims],
                  columns=['P_ref[eV]', 'P_pred[eV]', 'abs. error [eV]', 'Selected features'])

#  if you do not want to sort the data frame by the prediction error comment out the next line 
df_loo = df_loo.sort_values('abs. error [eV]', ascending=False)
pd.set_option('display.expand_frame_repr', False)

display(df_loo)

#but it's not that accurate of a prediction model... can be very wonky.

Descriptor selection frequency
-----------------
1D:
           Features  Frequency
0  (|EN(X)-EN(T)|,)        275
-----------------
2D:
                         Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)))        275
-----------------
3D:
                                        Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)))        236
1  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|)         39
-----------------
4D:
                                            Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...        236
1  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...         39
-----------------
5D:
                                            Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...        196
1  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...         40
2  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...         22
3  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...         17
---------------

P_ref[eV]  P_pred[eV]  abs. error [eV]                                  Selected features
270 1D   1.347678    3.687384         3.129689                                   (|EN(X)-EN(T)|,)
    2D   1.347678    3.897748         2.919325                     (|EN(X)-EN(T)|, (EN(M)+EN(T)))
39  3D   5.664074    5.598196         2.873292  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...
    2D   5.664074    5.594244         2.869341  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...
193 2D   5.759952    3.689281         2.833742                                   (|EN(X)-EN(T)|,)
...           ...         ...              ...                                                ...
123 1D   6.377828    6.020369         0.006719  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...
72  2D   2.324315    2.333895         0.006121  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...
205 1D   1.561346    4.727063         0.003466  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
125 1D   3.314447    5.215839         0.002146  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...
215 1D   3.947450    1.423338         0.000330  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...

[825 rows x 4 columns]

In [16]:
# K-fold cross-validation (this is messy. work in progress)  
n_compounds = len(P)
dimensions = range(1, 11)
features_count = [[] for i in range(10)]
P_predict = np.empty([len(dimensions), n_compounds])
list_K_1D = [] 
list_K_2D = [] 
list_K_3D = [] 
list_K_4D = [] 
list_K_5D = []
list_K_6D = []
list_K_7D = []
list_K_8D = []
list_K_9D = []
list_K_10D = []

#insert empty list here 

sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)
kfold = KFold() #i think the default setting is: kfold = KFold(n_splits=5, shuffle=False, random_state=None)

for indices_train, index_test in kfold.split(P):
    i_cv = index_test[0]
    print('%2s) Leave out %s: Ediff_ref = %.2f eV' 
          % (index_test[0]+1, chemical_formulas[i_cv], P[i_cv])) #chemical_formulas = df_D.index.tolist() (list of values ranging from 0 to 274)
        
    sisso.fit(D[indices_train], P[indices_train])
    sisso.print_models(features_list)    
    
    for dim in dimensions:      
        features = [features_list[i] for i in sisso.l0_selected_indices[dim - 1]] #list index out of range 
        predicted_value = sisso.predict(D[index_test], dim=dim)[0]
        
        features_count[dim-1].append( tuple(features) )        
        P_predict[dim-1, i_cv] = predicted_value

        if dim == 1: 
            list_K_1D.append(P[index_test]-predicted_value)

        if dim == 2: 
            list_K_2D.append(P[index_test]-predicted_value)

        if dim == 3: 
            list_K_3D.append(P[index_test]-predicted_value)

        if dim == 4: 
            list_K_4D.append(P[index_test]- predicted_value)

        if dim == 5: 
            list_K_5D.append(P[index_test]- predicted_value)

        if dim == 6: 
            list_K_6D.append(P[index_test]-predicted_value)

        if dim == 7: 
            list_K_7D.append(P[index_test]-predicted_value)

        if dim == 8: 
            list_K_8D.append(P[index_test]-predicted_value)

        if dim == 9: 
            list_K_9D.append(P[index_test]- predicted_value)

        if dim == 10: 
            list_K_10D.append(P[index_test]- predicted_value)
        
        print('Ediff_predicted(%sD) = %.3f eV' %(dim, predicted_value))
    print('-----') #unit for workfunction is eV. change that later. 


 1) Leave out 0: Ediff_ref = 2.27 eV
          RMSE            Model
1D:	1.410455	+ 1.342 |EN(X)-EN(T)| + 3.167
2D:	1.299196	+ 1.563 |EN(X)-EN(T)| + 0.562 (EN(M)+EN(T)) + 0.433
3D:	1.023251	+ 1.867 |EN(X)-EN(T)| + 1.425 (EN(M)+EN(T)) + 1.065 (EA(M)-EA(T)) - 2.295
4D:	0.900414	+ 2.098 |EN(X)-EN(T)| + 1.975 (EN(M)+EN(T)) + 1.533 (EA(M)-EA(T)) - 1.722 EA(M) - 3.660
5D:	0.805303	+ 2.243 |EN(X)-EN(T)| + 2.406 (EN(M)+EN(T)) + 1.842 (EA(M)-EA(T)) - 1.566 EA(M) - 1.237 (EN(M)-EN(X)) - 6.870
6D:	0.750189	+ 2.072 |EN(X)-EN(T)| + 2.542 (EN(M)+EN(T)) + 1.762 (EA(M)-EA(T)) - 1.258 EA(M) - 1.482 (EN(M)-EN(X)) - 0.021 |r-(M)-r-(T)| - 7.443
7D:	0.714959	+ 2.010 |EN(X)-EN(T)| + 2.802 (EN(M)+EN(T)) + 1.904 (EA(M)-EA(T)) - 1.079 EA(M) - 1.609 (EN(M)-EN(X)) - 0.027 |r-(M)-r-(T)| - 0.726 (EN(M)+EN(X)) - 5.311
8D:	0.668147	+ 1.957 |EN(X)-EN(T)| + 2.997 (EN(M)+EN(T)) + 1.859 (EA(M)-EA(T)) - 1.130 EA(M) - 1.821 (EN(M)-EN(X)) - 0.028 |r-(M)-r-(T)| - 1.014 (EN(M)+EN(X)) + 1.503 E_Hull - 5.636
9D:	0.623299	+ 1.9

### SISSO without E-hull (new + original operator, LOOCV & K-Fold)

In [17]:
#import Data
# selected_feature_list = ['r_s', 'r_p', 'r_d', 'EA', 'IP']
# selected_feature_list = ordered_selected_features[:7] 


df_2 = pd.read_csv("275_without_EHull.csv") #need to check its location later)
df_2.dropna(inplace=True)
df_2.columns

ordered_selected_features = ['HoF','Length(x)', 'r-(M)', 'r-(X)', 'r-(T)', 'IP(M)', 'IP(X)', 'IP(T)', 'EN(M)', 'EN(X)','EN(T)', 'EA(M)', 'EA(X)', 'EA(T)']
df_2[ordered_selected_features]

def get_data(selected_feature_list, allowed_operations):
    
    # extract work function and selected features from df 
    P = df_2['Workfunction'].values
    df_features = df_2.iloc[:, 5:][selected_feature_list] #select 5th column onwards 
    print("df_features.columns", df_features.columns)
    
    # derive new features using allowed_operations
    df_combined = combine_features(df=df_features, allowed_operations=allowed_operations) 
    return P, df_combined


selected_feature_list = df_2.columns[5:]
allowed_operations = ['+','-','|-|','/','^3','^2','sin','cos','tan'] #exp,division seems to be the problem. because zero value cannot. 
P, df_D = get_data(selected_feature_list, allowed_operations)
features_list = df_D.columns.tolist()
chemical_formulas = df_D.index.tolist()
D = df_D.values
# D=a[a != 0] #would this disrupt the result actually? 

features_list = df_D.columns.tolist()

#look at df_D for the created dataset

df_features.columns Index(['HoF', 'Length(x)', 'r-(M)', 'r-(X)', 'r-(T)', 'IP(M)', 'IP(X)',
       'IP(T)', 'EN(M)', 'EN(X)', 'EN(T)', 'EA(M)', 'EA(X)', 'EA(T)'],
      dtype='object')
Selected operations:
 ['+', '-', '|-|', '/', '^3', '^2', 'sin', 'cos', 'tan']
Number of total features generated: 1871


In [18]:
sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)

sisso.fit(D, P) #error 
sisso.print_models(features_list)


          RMSE            Model
1D:	1.315447	+ 69155185.087 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 3.002
2D:	1.246605	+ 64085357.870 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 46542.588 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 1.487
3D:	1.109086	+ 70935891.456 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 86611.520 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 0.609 (EA(M)-EA(T)) + 0.806
4D:	0.957609	+ 81141721.023 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 65124.428 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 0.988 (EA(M)-EA(T)) - 110174.781 (EN(X)-EN(T))/r-(M)^3 + 1.798
5D:	0.856623	+ 89469834.914 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 84601.168 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 1.248 (EA(M)-EA(T)) - 105492.661 (EN(X)-EN(T))/r-(M)^3 - 8222.979 cos(HoF)/(r-(X)+r-(T))^2 + 1.559
6D:	0.791940	+ 90414780.030 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 105098.672 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 1.415 (EA(M)-EA(T)) - 110238.151 (EN(X)-EN(T))/r-(M)^3 - 8905.959 cos(HoF)/(r-(X)+r-(T))^2 - 8730.763 EA(M)/(r-(M)+r-(T))^2 + 1.344
7D:	0.705776	+ 1072

In [19]:
# Leave-one-out cross-validation
n_compounds = len(P)
dimensions = range(1, 11) #(1,2,3,4)
features_count = [[] for i in range(10)] #what exactly does this do? 
P_predict = np.empty([len(dimensions), n_compounds]) #create a empty array with no input. (row*column)
list_1D = [] 
list_2D = [] 
list_3D = [] 
list_4D = [] 
list_5D = []
list_6D = []
list_7D = []
list_8D = []
list_9D = []
list_10D = []

sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)
loo = LeaveOneOut() #cross validation method 
#examples of loo. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html 


for indices_train, index_test in loo.split(P):
    i_cv = index_test[0] 
    print('%2s) Leave out %s: Ediff_ref = %.2f eV' 
          % (index_test[0]+1, chemical_formulas[i_cv], P[i_cv])) #this is called the printf format string. https://en.wikipedia.org/wiki/Printf_format_string
        
    sisso.fit(D[indices_train], P[indices_train])
    sisso.print_models(features_list)    
    
    for dim in dimensions:      
        features = [features_list[i] for i in sisso.l0_selected_indices[dim - 1]]
        predicted_value = sisso.predict(D[index_test], dim=dim)[0]
        
        features_count[dim-1].append(tuple(features))        
        P_predict[dim-1, i_cv] = predicted_value

        if dim == 1: 
            list_1D.append(P[index_test]-predicted_value)

        if dim == 2: 
            list_2D.append(P[index_test]-predicted_value)

        if dim == 3: 
            list_3D.append(P[index_test]-predicted_value)

        if dim == 4: 
            list_4D.append(P[index_test]- predicted_value)

        if dim == 5: 
            list_5D.append(P[index_test]- predicted_value)

        if dim == 6: 
            list_6D.append(P[index_test]-predicted_value)

        if dim == 7: 
            list_7D.append(P[index_test]-predicted_value)

        if dim == 8: 
            list_8D.append(P[index_test]-predicted_value)

        if dim == 9: 
            list_9D.append(P[index_test]- predicted_value)

        if dim == 10: 
            list_10D.append(P[index_test]- predicted_value)

        print('Ediff_predicted(%sD) = %.3f eV' %(dim, predicted_value))
    print('-----') #unit for workfunction is eV. change that later. 

for dimensions in dim: 
    sum()

 1) Leave out 0: Ediff_ref = 2.27 eV
          RMSE            Model
1D:	1.316173	+ 68741659.436 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 3.013
2D:	1.247903	+ 63801794.746 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 46321.042 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 1.502
3D:	1.111005	+ 70837842.732 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 86479.507 (EN(M)+EN(T))/(Length(x)+r-(X))^2 - 0.608 |EA(M)-EA(T)| + 0.811
4D:	0.959318	+ 81085071.540 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 65054.484 (EN(M)+EN(T))/(Length(x)+r-(X))^2 - 0.988 |EA(M)-EA(T)| - 110154.990 (EN(X)-EN(T))/r-(M)^3 + 1.801
5D:	0.858144	+ 89532499.088 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 84685.614 (EN(M)+EN(T))/(Length(x)+r-(X))^2 - 1.248 |EA(M)-EA(T)| - 105508.712 (EN(X)-EN(T))/r-(M)^3 - 8229.306 cos(HoF)/(r-(X)+r-(T))^2 + 1.556
6D:	0.793305	+ 90498074.376 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 105216.365 (EN(M)+EN(T))/(Length(x)+r-(X))^2 - 1.416 |EA(M)-EA(T)| - 110260.762 (EN(X)-EN(T))/r-(M)^3 - 8914.536 cos(HoF)/(r-(X)+r-(T))^2 - 8733.236 EA(M)/(r-(M)

TypeError: 'int' object is not iterable

In [20]:
list_1D = [((element)**2) for element in list_1D] #this is the equation for RMSE 
list_2D = [((element)**2) for element in list_2D] 
list_3D = [((element)**2) for element in list_3D] 
list_4D = [((element)**2) for element in list_4D] 
list_5D = [((element)**2) for element in list_5D] 
list_6D = [((element)**2) for element in list_6D] 
list_7D = [((element)**2) for element in list_7D] 
list_8D = [((element)**2) for element in list_8D] 
list_9D = [((element)**2) for element in list_9D] 
list_10D = [((element)**2) for element in list_10D] 

def Average(lst):
    return sum(lst) / len(lst)


print(Average(list_1D),Average(list_2D), Average(list_3D), Average(list_4D),Average(list_5D),Average(list_6D),Average(list_7D), Average(list_8D), Average(list_9D),Average(list_10D)) 
#rmse value for 1D-4D. 

[1.75427112] [1.60024641] [1.27952888] [1.21686782] [0.77090507] [0.67802059] [0.5535003] [0.45349574] [0.44902818] [0.41882862]


In [21]:
#Plot Prediction errors
prediction_errors = np.linalg.norm(P-P_predict, axis=1)/np.sqrt(n_compounds)
xs = [P] *4
ys = P_predict
legend = ['%sD, RMSE = %.4f eV' %(dim, prediction_errors[dim-1]) for dim in dimensions]
data_point_labels = [df_D.index.tolist()]*4

show_scatter_plot(xs, ys, data_point_labels=data_point_labels, 
                  x_label='WF_DFT', y_label='WF_predicted', legend=legend, unit='eV')

In [22]:
# Print descriptor selection frequency
print("Descriptor selection frequency")
for dim in dimensions:    
    df_frequency = pd.DataFrame( Counter(features_count[dim-1]).most_common(10), columns=['Features', 'Frequency'] )
    print('-----------------\n%sD:\n%s' % (dim, df_frequency))

# create table to display errors and models
feat = np.array(features_count).flatten('F')
Pred = np.array(P_predict).flatten('F')
Pred_errors = np.abs(P-P_predict).flatten('F')
Ref_values = [r for p in P for r in [p,p,p] ]
Mats = [m for mat in chemical_formulas for m in [mat, mat, mat] ]
Dims = ['1D','2D','3D'] * n_compounds

df_loo = pd.DataFrame(list(zip(Ref_values,Pred,Pred_errors,feat)), index = [Mats,Dims],
                  columns=['P_ref[eV]', 'P_pred[eV]', 'abs. error [eV]', 'Selected features'])

#  if you do not want to sort the data frame by the prediction error comment out the next line 
df_loo = df_loo.sort_values('abs. error [eV]', ascending=False)
pd.set_option('display.expand_frame_repr', False)

display(df_loo)

#but it's not that accurate of a prediction model... can be very wonky.

Descriptor selection frequency
-----------------
1D:
                               Features  Frequency
0  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3,)        275
-----------------
2D:
                                            Features  Frequency
0  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(...        266
1  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(...          9
-----------------
3D:
                                            Features  Frequency
0  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(...        134
1  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(...        132
2  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(...          5
3  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(...          4
-----------------
4D:
                                            Features  Frequency
0  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(...        114
1  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(...        112
2  (|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(...         20
3  (|EN(X)-

,,P_ref[eV],P_pred[eV],abs. error [eV],Selected features
270,1D,1.347678,3.860078,2.956995,"(|EN(X)-EN(T)|/(Length(x)+r-(T))^3,)"
273,2D,2.043231,3.544493,2.854250,"(|EN(X)-EN(T)|/(Length(x)+r-(T))^3,)"
274,1D,2.579342,3.573671,2.825072,"(|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(..."
273,3D,2.043231,3.585315,2.813428,"(|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(..."
193,2D,5.759952,3.762543,2.760480,"(|EN(X)-EN(T)|/(Length(x)+r-(T))^3,)"
...,...,...,...,...,...
231,2D,4.394605,2.397692,0.003714,"(|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(..."
87,2D,2.711951,4.466339,0.002776,"(|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(..."
121,3D,4.803881,6.015977,0.002326,"(|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(..."
43,1D,3.917393,5.538242,0.000921,"(|EN(X)-EN(T)|/(Length(x)+r-(T))^3, (EN(M)+EN(..."


In [23]:
# K-fold cross-validation (this is messy. work in progress)  
n_compounds = len(P)
dimensions = range(1, 11)
features_count = [[] for i in range(10)]
P_predict = np.empty([len(dimensions), n_compounds])
list_K_1D = [] 
list_K_2D = [] 
list_K_3D = [] 
list_K_4D = [] 
list_K_5D = []
list_K_6D = []
list_K_7D = []
list_K_8D = []
list_K_9D = []
list_K_10D = []

#insert empty list here 

sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)
kfold = KFold() #i think the default setting is: kfold = KFold(n_splits=5, shuffle=False, random_state=None)

for indices_train, index_test in kfold.split(P):
    i_cv = index_test[0]
    print('%2s) Leave out %s: Ediff_ref = %.2f eV' 
          % (index_test[0]+1, chemical_formulas[i_cv], P[i_cv])) #chemical_formulas = df_D.index.tolist() (list of values ranging from 0 to 274)
        
    sisso.fit(D[indices_train], P[indices_train])
    sisso.print_models(features_list)    
    
    for dim in dimensions:      
        features = [features_list[i] for i in sisso.l0_selected_indices[dim - 1]] #list index out of range 
        predicted_value = sisso.predict(D[index_test], dim=dim)[0]
        
        features_count[dim-1].append( tuple(features) )        
        P_predict[dim-1, i_cv] = predicted_value

        if dim == 1: 
            list_K_1D.append(P[index_test]-predicted_value)

        if dim == 2: 
            list_K_2D.append(P[index_test]-predicted_value)

        if dim == 3: 
            list_K_3D.append(P[index_test]-predicted_value)

        if dim == 4: 
            list_K_4D.append(P[index_test]- predicted_value)

        if dim == 5: 
            list_K_5D.append(P[index_test]- predicted_value)

        if dim == 6: 
            list_K_6D.append(P[index_test]-predicted_value)

        if dim == 7: 
            list_K_7D.append(P[index_test]-predicted_value)

        if dim == 8: 
            list_K_8D.append(P[index_test]-predicted_value)

        if dim == 9: 
            list_K_9D.append(P[index_test]- predicted_value)

        if dim == 10: 
            list_K_10D.append(P[index_test]- predicted_value)
        
        print('Ediff_predicted(%sD) = %.3f eV' %(dim, predicted_value))
    print('-----') #unit for workfunction is eV. change that later. 

for dimensions in dim: 
    sum() 



 1) Leave out 0: Ediff_ref = 2.27 eV
          RMSE            Model
1D:	1.340822	+ 69877859.925 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 2.992
2D:	1.275773	+ 64159273.852 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 45374.225 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 1.531
3D:	1.128931	+ 67088905.830 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 88773.854 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 0.659 (EA(M)-EA(T)) + 0.875
4D:	1.006794	+ 76077009.789 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 115047.149 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 1.003 (EA(M)-EA(T)) - 885791.792 cos(HoF)/(r-(X)+r-(T))^3 + 0.500
5D:	0.898006	+ 77864994.241 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 105528.517 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 1.277 (EA(M)-EA(T)) - 996766.693 cos(HoF)/(r-(X)+r-(T))^3 - 63829.374 (EN(M)-EN(T))/r-(M)^3 + 0.731
6D:	0.732703	+ 96692985.832 |EN(X)-EN(T)|/(Length(x)+r-(T))^3 + 150149.458 (EN(M)+EN(T))/(Length(x)+r-(X))^2 + 1.663 (EA(M)-EA(T)) - 687598.482 cos(HoF)/(r-(X)+r-(T))^3 - 113303.594 (EN(M)-EN(T))/r-(M)^3 + 0.000 (Length

TypeError: 'int' object is not iterable

### SISSO without E-hull (original operator, LOOCV & K-Fold)

In [24]:
#import Data
# selected_feature_list = ['r_s', 'r_p', 'r_d', 'EA', 'IP']
# selected_feature_list = ordered_selected_features[:7] 


df_2 = pd.read_csv("275_without_EHull.csv") #need to check its location later)
df_2.dropna(inplace=True)
df_2.columns

ordered_selected_features = ['HoF','Length(x)','r-(M)', 'r-(X)', 'r-(T)', 'IP(M)', 'IP(X)', 'IP(T)', 'EN(M)', 'EN(X)','EN(T)', 'EA(M)', 'EA(X)', 'EA(T)']
df_2[ordered_selected_features]

def get_data(selected_feature_list, allowed_operations):
    
    # extract work function and selected features from df 
    P = df_2['Workfunction'].values
    df_features = df_2.iloc[:, 5:][selected_feature_list] #select 5th column onwards 
    print("df_features.columns", df_features.columns)
    
    # derive new features using allowed_operations
    df_combined = combine_features(df=df_features, allowed_operations=allowed_operations) 
    return P, df_combined


selected_feature_list = df_2.columns[5:]
allowed_operations = ['+','-','|-|','^3','^2'] #exp,division seems to be the problem. because zero value cannot. 
P, df_D = get_data(selected_feature_list, allowed_operations)
features_list = df_D.columns.tolist()
chemical_formulas = df_D.index.tolist()
D = df_D.values
# D=a[a != 0] #would this disrupt the result actually? 

features_list = df_D.columns.tolist()

#look at df_D for the created dataset

df_features.columns Index(['HoF', 'Length(x)', 'r-(M)', 'r-(X)', 'r-(T)', 'IP(M)', 'IP(X)',
       'IP(T)', 'EN(M)', 'EN(X)', 'EN(T)', 'EA(M)', 'EA(X)', 'EA(T)'],
      dtype='object')
Selected operations:
 ['+', '-', '|-|', '^3', '^2']
Number of total features generated: 108


In [25]:
sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)

sisso.fit(D, P) #error 
sisso.print_models(features_list)


          RMSE            Model
1D:	1.387989	+ 1.321 |EN(X)-EN(T)| + 3.179
2D:	1.281586	+ 1.534 |EN(X)-EN(T)| + 0.550 (EN(M)+EN(T)) + 0.502
3D:	1.061405	+ 1.890 |EN(X)-EN(T)| + 1.174 (EN(M)+EN(T)) + 0.775 (IP(M)-EA(T)) - 6.683
4D:	0.865174	+ 2.143 |EN(X)-EN(T)| + 2.096 (EN(M)+EN(T)) + 1.607 (IP(M)-EA(T)) - 1.973 IP(M) - 1.665
5D:	0.808244	+ 2.245 |EN(X)-EN(T)| + 2.390 (EN(M)+EN(T)) + 1.806 (IP(M)-EA(T)) - 1.922 IP(M) - 0.953 (EN(M)-EN(X)) - 5.613
6D:	0.757767	+ 2.118 |EN(X)-EN(T)| + 2.514 (EN(M)+EN(T)) + 1.718 (IP(M)-EA(T)) - 1.831 IP(M) - 1.121 (EN(M)-EN(X)) - 0.020 |r-(M)-r-(T)| - 6.063
7D:	0.715868	+ 1.986 |EN(X)-EN(T)| + 2.824 (EN(M)+EN(T)) + 1.750 (IP(M)-EA(T)) - 1.877 IP(M) - 1.503 (EN(M)-EN(X)) - 0.019 |r-(M)-r-(T)| + 0.000 (r-(X)+r-(T))^3 - 8.418
8D:	0.690959	+ 1.953 |EN(X)-EN(T)| + 2.840 (EN(M)+EN(T)) + 1.765 (IP(M)-EA(T)) - 2.089 IP(M) - 1.847 (EN(M)-EN(X)) - 0.008 |r-(M)-r-(T)| + 0.000 (r-(X)+r-(T))^3 - 0.016 |Length(x)-r-(X)| - 3.943
9D:	0.677353	+ 1.923 |EN(X)-EN(T)| + 2.9

In [26]:
# Leave-one-out cross-validation
n_compounds = len(P)
dimensions = range(1, 11) #(1,2,3,4)
features_count = [[] for i in range(10)] #what exactly does this do? 
P_predict = np.empty([len(dimensions), n_compounds]) #create a empty array with no input. (row*column)
list_1D = [] 
list_2D = [] 
list_3D = [] 
list_4D = [] 
list_5D = []
list_6D = []
list_7D = []
list_8D = []
list_9D = []
list_10D = []

sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)
loo = LeaveOneOut() #cross validation method 
#examples of loo. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html 


for indices_train, index_test in loo.split(P):
    i_cv = index_test[0] 
    print('%2s) Leave out %s: Ediff_ref = %.2f eV' 
          % (index_test[0]+1, chemical_formulas[i_cv], P[i_cv])) #this is called the printf format string. https://en.wikipedia.org/wiki/Printf_format_string
        
    sisso.fit(D[indices_train], P[indices_train])
    sisso.print_models(features_list)    
    
    for dim in dimensions:      
        features = [features_list[i] for i in sisso.l0_selected_indices[dim - 1]]
        predicted_value = sisso.predict(D[index_test], dim=dim)[0]
        
        features_count[dim-1].append(tuple(features))        
        P_predict[dim-1, i_cv] = predicted_value

        if dim == 1: 
            list_1D.append(P[index_test]-predicted_value)

        if dim == 2: 
            list_2D.append(P[index_test]-predicted_value)

        if dim == 3: 
            list_3D.append(P[index_test]-predicted_value)

        if dim == 4: 
            list_4D.append(P[index_test]- predicted_value)

        if dim == 5: 
            list_5D.append(P[index_test]- predicted_value)

        if dim == 6: 
            list_6D.append(P[index_test]-predicted_value)

        if dim == 7: 
            list_7D.append(P[index_test]-predicted_value)

        if dim == 8: 
            list_8D.append(P[index_test]-predicted_value)

        if dim == 9: 
            list_9D.append(P[index_test]- predicted_value)

        if dim == 10: 
            list_10D.append(P[index_test]- predicted_value)

        print('Ediff_predicted(%sD) = %.3f eV' %(dim, predicted_value))
    print('-----') #unit for workfunction is eV. change that later. 

for dimensions in dim: 
    sum()

 1) Leave out 0: Ediff_ref = 2.27 eV
          RMSE            Model
1D:	1.388402	+ 1.310 |EN(X)-EN(T)| + 3.193
2D:	1.282737	+ 1.525 |EN(X)-EN(T)| + 0.547 (EN(M)+EN(T)) + 0.523
3D:	1.063218	+ 1.887 |EN(X)-EN(T)| + 1.172 (EN(M)+EN(T)) + 0.774 |IP(M)-EA(T)| - 6.668
4D:	0.866701	+ 2.145 |EN(X)-EN(T)| + 2.098 (EN(M)+EN(T)) + 1.608 |IP(M)-EA(T)| - 1.974 IP(M) - 1.670
5D:	0.809459	+ 2.249 |EN(X)-EN(T)| + 2.393 (EN(M)+EN(T)) + 1.809 |IP(M)-EA(T)| - 1.925 IP(M) - 0.955 (EN(M)-EN(X)) - 5.635
6D:	0.757820	+ 2.126 |EN(X)-EN(T)| + 2.524 (EN(M)+EN(T)) + 1.723 |IP(M)-EA(T)| - 1.835 IP(M) - 1.128 (EN(M)-EN(X)) - 0.020 |r-(M)-r-(T)| - 6.116
7D:	0.715566	+ 1.995 |EN(X)-EN(T)| + 2.836 (EN(M)+EN(T)) + 1.755 |IP(M)-EA(T)| - 1.882 IP(M) - 1.512 (EN(M)-EN(X)) - 0.019 |r-(M)-r-(T)| + 0.000 (r-(X)+r-(T))^3 - 8.481
8D:	0.690823	+ 1.960 |EN(X)-EN(T)| + 2.851 (EN(M)+EN(T)) + 1.770 |IP(M)-EA(T)| - 2.092 IP(M) - 1.853 (EN(M)-EN(X)) - 0.009 |r-(M)-r-(T)| + 0.000 (r-(X)+r-(T))^3 - 0.016 |Length(x)-r-(X)| - 4.023
9D:

TypeError: 'int' object is not iterable

In [27]:
list_1D = [((element)**2) for element in list_1D] #this is the equation for RMSE 
list_2D = [((element)**2) for element in list_2D] 
list_3D = [((element)**2) for element in list_3D] 
list_4D = [((element)**2) for element in list_4D] 
list_5D = [((element)**2) for element in list_5D] 
list_6D = [((element)**2) for element in list_6D] 
list_7D = [((element)**2) for element in list_7D] 
list_8D = [((element)**2) for element in list_8D] 
list_9D = [((element)**2) for element in list_9D] 
list_10D = [((element)**2) for element in list_10D] 

def Average(lst):
    return sum(lst) / len(lst)


print(Average(list_1D),Average(list_2D), Average(list_3D), Average(list_4D),Average(list_5D),Average(list_6D),Average(list_7D), Average(list_8D), Average(list_9D),Average(list_10D)) 
#rmse value for 1D-4D. 

[1.95411303] [1.67131877] [1.15281979] [0.77552334] [0.68398583] [0.62861485] [0.58001735] [0.5426671] [0.51866477] [0.46208068]


In [28]:
#Plot Prediction errors
prediction_errors = np.linalg.norm(P-P_predict, axis=1)/np.sqrt(n_compounds)
xs = [P] *4
ys = P_predict
legend = ['%sD, RMSE = %.4f eV' %(dim, prediction_errors[dim-1]) for dim in dimensions]
data_point_labels = [df_D.index.tolist()]*4

show_scatter_plot(xs, ys, data_point_labels=data_point_labels, 
                  x_label='WF_DFT', y_label='WF_predicted', legend=legend, unit='eV')

In [29]:
# Print descriptor selection frequency
print("Descriptor selection frequency")
for dim in dimensions:    
    df_frequency = pd.DataFrame( Counter(features_count[dim-1]).most_common(10), columns=['Features', 'Frequency'] )
    print('-----------------\n%sD:\n%s' % (dim, df_frequency))

# create table to display errors and models
feat = np.array(features_count).flatten('F')
Pred = np.array(P_predict).flatten('F')
Pred_errors = np.abs(P-P_predict).flatten('F')
Ref_values = [r for p in P for r in [p,p,p] ]
Mats = [m for mat in chemical_formulas for m in [mat, mat, mat] ]
Dims = ['1D','2D','3D'] * n_compounds

df_loo = pd.DataFrame(list(zip(Ref_values,Pred,Pred_errors,feat)), index = [Mats,Dims],
                  columns=['P_ref[eV]', 'P_pred[eV]', 'abs. error [eV]', 'Selected features'])

#  if you do not want to sort the data frame by the prediction error comment out the next line 
df_loo = df_loo.sort_values('abs. error [eV]', ascending=False)
pd.set_option('display.expand_frame_repr', False)

display(df_loo)

#but it's not that accurate of a prediction model... can be very wonky.

Descriptor selection frequency
-----------------
1D:
           Features  Frequency
0  (|EN(X)-EN(T)|,)        275
-----------------
2D:
                         Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)))        275
-----------------
3D:
                                        Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)))        215
1  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|)         60
-----------------
4D:
                                            Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...        215
1  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...         60
-----------------
5D:
                                            Features  Frequency
0  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...        115
1  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...        100
2  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...         35
3  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...         25
---------------

P_ref[eV]  P_pred[eV]  abs. error [eV]                                  Selected features
270 1D   1.347678    3.687384         3.129689                                   (|EN(X)-EN(T)|,)
    2D   1.347678    3.897748         2.919325                     (|EN(X)-EN(T)|, (EN(M)+EN(T)))
193 2D   5.759952    3.689281         2.833742                                   (|EN(X)-EN(T)|,)
38  1D   5.816441    5.519206         2.794302  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
37  3D   5.213693    5.476533         2.751629  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
...           ...         ...              ...                                                ...
156 2D   1.917367    4.342918         0.003269  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...
259 1D   4.515988    6.144199         0.002162  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...
125 1D   3.314447    5.215839         0.002146  (|EN(X)-EN(T)|, (EN(M)+EN(T)), (IP(M)-EA(T)), ...
132 1D   2.030781    5.665139         0.001065  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...
215 1D   3.947450    1.423338         0.000330  (|EN(X)-EN(T)|, (EN(M)+EN(T)), |IP(M)-EA(T)|, ...

[825 rows x 4 columns]

In [30]:
# K-fold cross-validation (this is messy. work in progress)  
n_compounds = len(P)
dimensions = range(1, 11)
features_count = [[] for i in range(10)]
P_predict = np.empty([len(dimensions), n_compounds])
list_K_1D = [] 
list_K_2D = [] 
list_K_3D = [] 
list_K_4D = [] 
list_K_5D = []
list_K_6D = []
list_K_7D = []
list_K_8D = []
list_K_9D = []
list_K_10D = []

#insert empty list here 

sisso = SissoRegressor(n_nonzero_coefs=10, n_features_per_sis_iter=1)
kfold = KFold() #i think the default setting is: kfold = KFold(n_splits=5, shuffle=False, random_state=None)

for indices_train, index_test in kfold.split(P):
    i_cv = index_test[0]
    print('%2s) Leave out %s: Ediff_ref = %.2f eV' 
          % (index_test[0]+1, chemical_formulas[i_cv], P[i_cv])) #chemical_formulas = df_D.index.tolist() (list of values ranging from 0 to 274)
        
    sisso.fit(D[indices_train], P[indices_train])
    sisso.print_models(features_list)    
    
    for dim in dimensions:      
        features = [features_list[i] for i in sisso.l0_selected_indices[dim - 1]] #list index out of range 
        predicted_value = sisso.predict(D[index_test], dim=dim)[0]
        
        features_count[dim-1].append( tuple(features) )        
        P_predict[dim-1, i_cv] = predicted_value

        if dim == 1: 
            list_K_1D.append(P[index_test]-predicted_value)

        if dim == 2: 
            list_K_2D.append(P[index_test]-predicted_value)

        if dim == 3: 
            list_K_3D.append(P[index_test]-predicted_value)

        if dim == 4: 
            list_K_4D.append(P[index_test]- predicted_value)

        if dim == 5: 
            list_K_5D.append(P[index_test]- predicted_value)

        if dim == 6: 
            list_K_6D.append(P[index_test]-predicted_value)

        if dim == 7: 
            list_K_7D.append(P[index_test]-predicted_value)

        if dim == 8: 
            list_K_8D.append(P[index_test]-predicted_value)

        if dim == 9: 
            list_K_9D.append(P[index_test]- predicted_value)

        if dim == 10: 
            list_K_10D.append(P[index_test]- predicted_value)
        
        print('Ediff_predicted(%sD) = %.3f eV' %(dim, predicted_value))
    print('-----') #unit for workfunction is eV. change that later. 

for dimensions in dim: 
    sum() 



 1) Leave out 0: Ediff_ref = 2.27 eV
          RMSE            Model
1D:	1.410455	+ 1.342 |EN(X)-EN(T)| + 3.167
2D:	1.299196	+ 1.563 |EN(X)-EN(T)| + 0.562 (EN(M)+EN(T)) + 0.433
3D:	1.023251	+ 1.867 |EN(X)-EN(T)| + 1.425 (EN(M)+EN(T)) + 1.065 (EA(M)-EA(T)) - 2.295
4D:	0.900414	+ 2.098 |EN(X)-EN(T)| + 1.975 (EN(M)+EN(T)) + 1.533 (EA(M)-EA(T)) - 1.722 EA(M) - 3.660
5D:	0.805303	+ 2.243 |EN(X)-EN(T)| + 2.406 (EN(M)+EN(T)) + 1.842 (EA(M)-EA(T)) - 1.566 EA(M) + 1.237 |EN(M)-EN(X)| - 6.870
6D:	0.750189	+ 2.072 |EN(X)-EN(T)| + 2.542 (EN(M)+EN(T)) + 1.762 (EA(M)-EA(T)) - 1.258 EA(M) + 1.482 |EN(M)-EN(X)| - 0.021 |r-(M)-r-(T)| - 7.443
7D:	0.714959	+ 2.010 |EN(X)-EN(T)| + 2.802 (EN(M)+EN(T)) + 1.904 (EA(M)-EA(T)) - 1.079 EA(M) + 1.609 |EN(M)-EN(X)| - 0.027 |r-(M)-r-(T)| - 0.726 (EN(M)+EN(X)) - 5.311
8D:	0.690251	+ 1.943 |EN(X)-EN(T)| + 2.869 (EN(M)+EN(T)) + 1.904 (EA(M)-EA(T)) - 1.170 EA(M) + 1.764 |EN(M)-EN(X)| - 0.032 |r-(M)-r-(T)| - 1.086 (EN(M)+EN(X)) - 0.014 (Length(x)-r-(M)) - 0.508
9D:	0.6

TypeError: 'int' object is not iterable